In [2]:
# Cell 1: Import Libraries and Set API Keys

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI # For interacting with OpenAI's LLMs
from langchain.agents import AgentExecutor, create_react_agent # Core components for building agents
from langchain_core.prompts import PromptTemplate # For defining how the LLM should respond
from langchain_community.utilities import SerpAPIWrapper # For using SerpAPI to perform web searches
from langchain.tools import Tool # A general wrapper to make functions usable by the agent
from langchain_core.messages import HumanMessage, AIMessage # For managing conversation history

# --- Set your API Keys from .env ---
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
serp_api_key = os.getenv('SERPAPI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

if serp_api_key:
    print(f"Serp API Key exists and begins {serp_api_key[:8]}")
else:
    print("Serp API Key not set")
   
print("All necessary libraries imported and API Key environment variables should be set.")




# Cell 2: Define the Agent's Tool(s)

# Initialize the SerpAPIWrapper. This object knows how to make requests to SerpAPI.
try:
    search = SerpAPIWrapper()
    # Define a list of tools that our agent can use.
    # Each tool is represented by a LangChain 'Tool' object.
    tools = [
        Tool(
            name="Search", # A unique, descriptive name for the tool. The LLM will refer to it by this name.
            func=search.run, # The Python function that gets called when this tool is used (e.g., search.run('query')).
            description="""
            Useful for when you need to answer questions about current events,
            up-to-date facts, or when you need to get factual information that
            might change over time. Input should be a concise question for the search engine.
            """ # This description is CRITICAL. The LLM reads this to decide WHEN to use the tool.
        )
    ]
    print("Search tool (SerpAPI) initialized successfully.")
except Exception as e:
    print(f"Error initializing SerpAPI: {e}.")
    print("Please ensure your SERPAPI_API_KEY is correctly set in Cell 1 and is valid.")
    print("The agent might not be able to perform web searches without a working SerpAPI key.")
    tools = [] # If SerpAPI fails, we set tools to an empty list so the program can still run (but with limited functionality).







# Cell 3: Initialize the Large Language Model (LLM)

# We initialize a ChatOpenAI model, which is designed for conversational interactions.
# We're using "gpt-4o", a powerful and versatile model by OpenAI.
# `temperature=0` makes the model's responses more deterministic and factual,
# which is generally preferred for agents performing specific tasks, reducing 'hallucinations'.
try:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    print("LLM (ChatOpenAI - gpt-4o-mini) initialized successfully.")
except Exception as e:
    print(f"Error initializing OpenAI LLM: {e}.")
    print("Please ensure your OPENAI_API_KEY is correctly set in Cell 1 and is valid, and you have an active internet connection.")
    print("Cannot proceed without a working LLM.")
    llm = None # Set LLM to None if initialization fails, to prevent further errors.





# Cell 4: Create the Agent and its Prompt Template

# Define the agent's prompt template.
# This template is passed to the LLM and guides its reasoning process.
# It explicitly lists the tools, defines the expected "Thought/Action/Observation" format,
# and includes placeholders for user input and chat history.
prompt_template = PromptTemplate.from_template("""
You are a helpful and intelligent AI assistant.
If the answer is immediately known without needing a tool, provide the Final Answer directly.
Otherwise, you have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, and if you need to use a tool.
Action: the action to take, should be one of [{tool_names}] 
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times if needed)
Thought: I now know the final answer or have gathered enough information.
Final Answer: the final, concise answer to the original input question
Begin!
Previous conversation history:
{chat_history}
{agent_scratchpad}
Question: {input} 
Thought: 
""")

# Create the agent itself using the ReAct (Reasoning + Acting) paradigm.
# `create_react_agent` is a LangChain helper that sets up the LLM to follow the ReAct prompt structure.
# It takes the LLM, the list of tools, and the prompt template as input.
if llm and tools: # Ensure LLM and tools are successfully initialized before creating the agent
    agent = create_react_agent(llm, tools, prompt_template)
    print("Agent created with ReAct paradigm and available tools.")
else:
    print("Warning: Agent could not be fully created due in Cell 3 (missing LLM) or Cell 2 (missing tools).")
    # As a fallback, if LLM is available but tools are not, we create a basic LLMChain.
    # This agent won't be able to use tools, but can still answer from its base knowledge.
    if llm:
        from langchain.chains import LLMChain
        from langchain.prompts import ChatPromptTemplate
        simple_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI assistant. Answer questions based on your internal knowledge."),
            ("human", "{input}")
        ])
        agent = LLMChain(llm=llm, prompt=simple_prompt)
        print("Agent created as a basic LLMChain (no tool use functionality).")
    else:
        agent = None # Agent cannot be created without an LLM.




# Cell 5: Create the Agent Executor and Initialize Chat History

# The AgentExecutor is responsible for running the agent.
# It takes the agent itself and the list of available tools.
# `verbose=True` is extremely useful for development, as it prints out the agent's
# internal 'Thought', 'Action', 'Action Input', and 'Observation' steps.
# `handle_parsing_errors=True` helps the agent recover if the LLM doesn't perfectly
# follow the ReAct output format (e.g., misspellings).
# `max_iterations=5` sets a limit on how many Thought/Action/Observation steps the
# agent can take before giving up, preventing infinite loops.
if agent and tools: # Ensure agent and tools were successfully created in previous cells
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, max_iterations=5)
    print("Agent Executor created. Ready to run queries!")
else:
    print("Agent Executor not created as agent or tools are not properly initialized. Cannot run queries.")
    agent_executor = None # Set to None if initialization failed.

# Initialize an empty list to store our conversation history.
# This list will be updated with each user query and agent response,
# allowing the agent to maintain context across multiple turns.
chat_history = []

print("Chat history initialized.")



# Cell 6: Define a Helper Function to Run Queries

# This function will encapsulate the logic for sending a query to the agent
# and managing the conversation history.
def run_agent_query(query: str):
    global chat_history # We use 'global' to indicate that we want to modify the chat_history list defined outside this function.

    if agent_executor: # Ensure the executor was successfully initialized
        print(f"\n--- User Query: {query} ---")
        try:
            # Invoke the agent executor.
            # We pass the current user 'input' and the 'chat_history' for context.
            response = agent_executor.invoke({"input": query, "chat_history": chat_history})
            # Extract the final answer from the response dictionary.
            final_answer = response.get('output', "No output generated.")

            print(f"\nAgent's Final Answer: {final_answer}")

            # Update the chat history for the next turn.
            # We append a HumanMessage (our query) and an AIMessage (the agent's response).
            chat_history.append(HumanMessage(content=query))
            chat_history.append(AIMessage(content=final_answer))

            return final_answer
        except Exception as e:
            print(f"An error occurred during agent execution: {e}")
            return "Sorry, I encountered an error trying to process your request."
    else:
        print("Agent Executor is not initialized. Cannot run query.")
        return "Agent is not ready to respond."







OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key not set
Serp API Key exists and begins 94a308d7
All necessary libraries imported and API Key environment variables should be set.
Search tool (SerpAPI) initialized successfully.
LLM (ChatOpenAI - gpt-4o-mini) initialized successfully.
Agent created with ReAct paradigm and available tools.
Agent Executor created. Ready to run queries!
Chat history initialized.


In [3]:
# Cell 7: Test Your Agent with Example Queries

print("\n--- Running Agent for Example Queries ---")

# --- Query 1: A simple factual question the LLM might know directly ---
# The agent should respond quickly without needing the search tool.
run_agent_query("What is the capital of India?")

# --- Query 2: A factual question requiring up-to-date information ---
# The agent should recognize it needs current data and use the 'Search' tool.
run_agent_query("As of year 2025, What is the population of Houston, Texas?")

# --- Query 3: A question leveraging previous context ---
# The agent should understand "this city" refers to "Houston" from the previous turn.
run_agent_query("What are the major sports teams in this city?")

# --- Query 4: A question about recent events ---
# This will definitely trigger the 'Search' tool to find the latest information.
run_agent_query("What is the latest news on AI safety regulations as of today?")

print("\n--- All example interactions complete ---")
print("\nFinal Full Chat History:")
# Loop through the entire chat_history to see the full conversation flow.
for msg in chat_history:
    print(f"{type(msg).__name__}: {msg.content}")


--- Running Agent for Example Queries ---

--- User Query: What is the capital of India? ---


> Entering new AgentExecutor chain...
The capital of India is New Delhi. 
Final Answer: New Delhi

> Finished chain.

Agent's Final Answer: New Delhi

--- User Query: As of year 2025, What is the population of Houston, Texas? ---


> Entering new AgentExecutor chain...
I need to find the most current population estimate for Houston, Texas, as of the year 2025. 
Action: Search 
Action Input: "Houston Texas population estimate 2025" Houston's population in 2025 is 2,319,119, with a metro area projection of 6.89 million. The city leads in STEM degrees, with 49.1% of graduates specializing in science and engineering. Harris County cut migration losses by half during the pandemic, losing 16,000 residents annually by 2022.Final Answer: The population of Houston, Texas, in 2025 is estimated to be 2,319,119.

> Finished chain.

Agent's Final Answer: The population of Houston, Texas, in 2025 is estim